Generalized World Coordinate System (GWCS)
==========================================


This section highlights one of the most powerful capabilities of ASDF; namely
its ability to save arbitrarily complex coordinate transformations with great 
flexibility. This is accomplished through an extension which serializes 
[GWCS](https://gwcs.readthedocs.io/en/latest/) objects. This notebook will show
the ability to:

- combine transformations in series or using arithmetic operators
- WCS for an arbitrary number of dimensions
- define intermediate coordinates (e.g., the slit plane of a spectrograph)
- parameterize transformations using the parameters as extra dimensions
  (e.g., spectral order, position across a slit, date, etc.)

Contrast this with the FITS WCS system, which works well in imaging and spectra
for standard projections and dispersions, but poorly when dealing with raw data
cointaining complex distortions, or discontinuous transforms (e.g., IFUs), and 
particularly for slitless spectroscopy.

For HST, to achieve sub 0.01 pixel accuracy, 3 different distortion components 
had to be modeled, which were impossible to represent within the FITS WCS 
framework.

We are not able to convey the full capabilities in a few minutes of a tutorial.
This tutorial will illustrate some basics with an imaging example.

### Imaging example

We will start with a simple projection and then augment with a distortion model.

The simple projection replicates the basic FITS capabilities using a tangent
projection followed by the appropriate transformation to celestial coordinates.

This involves identifying the point in the detector array that will be the tangent
point, applying the appropriate offset and scaling before applying the tangent
projection, and then transforming the resulting angular coordinates to celestial
coordinates. Schematically:

- Offset detector coordinates to make tangent point in detector have 0, 0 coordinates
- Scale resulting array coordinates to corresponding angular scale.
- Rotate detector coordinates so that north is up
- Apply inverse tangent projection.
- Transform resulting spherical coordinates to corresponding reference point
  in the celestial coordinate system with the appropriate position angle.

These operations are performed using astropy modeling package models.

In [ ]:
import asdf
import numpy as np
from astropy.modeling import models
from astropy import coordinates as coord
from astropy import units as u

from gwcs import wcs
from gwcs import coordinate_frames as cf

from matplotlib import pyplot as plt
%matplotlib widget 

For simplicity we will assume that the detector y-axis is aligned with north, so no 
rotation of detector coordinates is necessary.
First step is to define individual transformation models.
We assume the detector array is 2000 x 2000 and the tangent point is at (1000, 1000).

In [ ]:
# The following constructs a 2D model that shifts both input x and y coordinates by 1000
shift = models.Shift(-1000) & models.Shift(-1000)

# The following constructs a 2D model that scales both input x and y coordinates
# such that the center pixel is 0.1 arcsec in size
scale = models.Scale(0.1 / 3600.) & models.Scale(0.1 / 3600.)

# The following applies an inverse tangent projection
tanproj = models.Pix2Sky_TAN()

# The following moves the spherical coordinates so that the (0, 0) coordinates are moved
# to the supplied RA & Dec coordinates (in degrees), in this case RA = 30, Dec = 45
celest_rot = models.RotateNative2Celestial(30., 45., 180.) # last arg is always 180. deg for a gnomonic projection

# The following is the net transformation from pixel coordinates to celestial coordinates
transform = shift | scale | tanproj | celest_rot
transform.name = "to_sky"

In [ ]:
print(transform)

In [ ]:
# Now we define the frames of reference for the WCS
detector_frame = cf.Frame2D(name='detector', axes_names=('x','y'), unit=(u.pix, u.pix))
sky_frame = cf.CelestialFrame(reference_frame=coord.ICRS(), unit=(u.deg, u.deg))

A GWCS pipeline is a list of tuples: (coordinate_frame, transform to next frame). The last transform is `None`.

In [ ]:
wcsobj = wcs.WCS([(detector_frame, transform), 
                  (sky_frame, None)])

In [ ]:
print(wcsobj)

There are several ways to evaluate a GWCS object. The legacy code evaluates the WCS by calling it like a function.

In [ ]:
wcsobj(1000, 1000)

In [ ]:
wcsobj(1000, 1001)

In [ ]:
1/3600,

#### Shared WCS API

The Shared WCS API allows applications to work with any WCS library that supports it. The API includes methods returning high level astropy objects or numerical values.

- **pixel_to_world**, **world_to_pixel**: work with high level astropy objects
- **pixel_to_world_values**, **world_to_pixel_values**: numbers only

In [ ]:
sky = wcsobj.pixel_to_world(1000, 1000)
print(sky)

In [ ]:
wcsobj.world_to_pixel(sky)

In [ ]:
wcsobj.pixel_to_world_values(1000, 1000)

In [ ]:
wcsobj(1000, 1000)

Now let's save the WCS object to a file.

In [ ]:
af = asdf.AsdfFile()
af['wcs'] = wcsobj
af.write_to('wcs.asdf')
af.close()

#### Exercise: Add polynomial distortion to the imaging example

In this exercise we'll add polynomial distortion in `x` and `y`.

##### Directions

##### A 2D polynomial model is instantiated using
  
  
    from astropy.modeling import models
    p = models.Polynomial2D(degree=1, c0_0=.3)
  
To see the parameter names execute

    models.Polynomial2D(degree=1).param_names

  
Generate 2 polynomials in `x` and `y`  to correct the distortion on each axis: `px` and `py`.
  
##### Generate a combine model using
  

    distortion = models.Mapping((0, 1, 0, 1) | px & py

  
##### Folowing the imaging example above add this to the WCS pipeline as a first step, creating an intermediate frame of "undistorted coordinates"

  
    from gwcs import coordinate_frames as cf
      
    undistorted = cf.Frame2D(name='undistorted', unit=(u.pix, u.pix))
  
  

### Slitless spectroscopy example

For this example we will use a JWST NIRISS slitless observation. We will not follow the generation of the WCS but show some of its functionality, not available in FITS WCS. 

The JWST data products are decsribed by data models, built on top of ASDF, as an ASDF extension. These datamodels define custom tags. In order to open the file we need the JWST data models ASDF extension, called `stdatamodels`, installed.

This examples shows the following features

- Open a catalog of objects represented by an astropy table and saved in an asdf file.
- Open and display a grism image
- Choose one object from the catalog, find its position in the grism image and evaluate the WCS to compute the wavelength for different spectral orders
- Save the WCS for this particular object to a file.


In [ ]:
grism = asdf.open('../data/grism.asdf')

In [ ]:
grism.info()

In [ ]:
wgrism = grism['meta']['wcs']
grism_data = grism['data']


In [ ]:
print(wgrism)

In [ ]:
# Open the catalog
with asdf.open('../data/catalog.asdf') as af:
    cat = af['catalog']

print(cat[:5])

In [ ]:
cat.colnames

In [ ]:
# Choose one object in the catalog and match it to its location in the grism image.
obj_id = 541
obj_row = cat[obj_id]


What inputs and outputs are expected by the WCS pipeline?

The inputs are (x, y, x0, y0, order), where (x, y) are the coordinates in the grism image, (x0, y0) are coordinates of the object in the direct image, and the spectral order.

The output of the WCS pipeline are RA, DEC, wavelength and the spectral order is returned for reference.

In [ ]:
print(f"Inputs: {wgrism.get_transform('grism_detector', 'detector').inputs}")
print(f"Outputs: {wgrism.get_transform('v2v3vacorr', 'world').outputs}")

Next, read the center RA, DEC of the object from the catalog, and use the WCS to find its location in the grism image. We will use the mean of the wavelength range (defined in microns). Plot the grism image and a box around the slit as identified. 

In [ ]:
ra_center, dec_center = obj_row['sky_centroid'].ra.value, obj_row['sky_centroid'].dec.value
lmin, lmax = 1.7, 2.28
lam = (lmin + lmax) / 2
wgrism.invert(ra_center, dec_center, lam, 1)

In [ ]:
from matplotlib.patches import Rectangle

box = Rectangle((660, 750), 35, 150, linewidth=1, edgecolor='r', facecolor='none')
fig, ax = plt.subplots()
ax.imshow(grism_data, origin='lower', aspect='auto', vmin=-.1, vmax=2.3)
ax.add_patch(box)

In [ ]:
# The "x0", "y0" inputs refer to the coordinates of the object in the direct image associated with the grism observation.
# These are the "xcentroid" and "ycentroid" quantities in the catalog

x0, y0 = obj_row['xcentroid'], obj_row['ycentroid']
print(f"x0 = {x0}, y0 = {y0}")

In [ ]:
# We are ready now to evaluate the WCS
x, y = 680, 815
print(wgrism(x, y, x0, y0, order=1))


GWCS has a feature which allows inputs to be fixed to a certain value, returning a valid WCS for these specific values.

We will extract the WCS for this object and save it to a file for future use by fixing the coordinates of the object in the direct image. Inputs can be fixed by argument name or by position.

In [ ]:
w541 = wgrism.fix_inputs({2: x0, 3: y0})

In [ ]:
w541(x, y, 1)

In [ ]:
af = asdf.AsdfFile(tree={'wcs_541': w541})
af.write_to('wcs_541.asdf')

In [ ]:

with asdf.open('wcs_541.asdf') as newaf:
    wcs541 = newaf['wcs_541']

print(wcs541(x, y, 1))

Important Points
----------------

- All the defined transforms and GWCS objects (general and source-specific) are easily
  serialized to ASDF.
- This illustrates that one can define general transforms for all possible source positions
  that can be stored in the data file. 
- When the source positions are identified (perhaps later by more analysis or merging
  imaging data) specific transformations can be obtained from the general
  transforms easily without repeating a possibly complex general transform (e.g.,
  optical distortions can be folded into the tranforms), for each source identified
- Instead of using wavelength tables, one for each source identified, a common transform
  is available, and may be tweaked in one place (if not the source position) and
  automatically applied to all source dispersion transforms.
- The wavelength table approach is generated elsewhere; changes require rerunning
  the software that generated it whereas it is easy to generate a wavelength table
  from the transforms.
- The general slitless approach is also suitable for multiple object spectrographs
  and IFUs where slits may be located at arbitrary locations in the focal plane, such as
  those that use shutters such as the JWST NIRSpec MOS mode or the NIRSpec IFU.
- The ability to define transforms that fold in parameters as extra coordinates
  that may vary between datasets such as order, temperature, date, etc. allows
  use of a single GWCS model that can be applied to all such variations without
  the need to store different GWCS models for each data set.

#### Exercise 2: Generate high level objects and transform to different coordinate systems

The common WCS API methods do not accept keyword arguments. In order to generate high level objects we will fix the spectral order.

- Generate a WCS object for the same object and fix the spectral order to -1
- Call the high level "pixel_to_world" method and inspect the output
- Transform the sky object to galactic coordinates
- Transform the spectral coordinate to Angstroms

### Major features we didn't cover

- Validation using jsonschema
- Versioning
- Chunking support using zarr